### Carga de librerías y revisión de datasets

In [4]:
# Importamos todas las librerias que utilizaremos en el proceo de ETL.
import numpy as np
import pandas as pd
import ast
import warnings
warnings.filterwarnings('ignore')


'\nimport seaborn as sns\nimport matplotlib.pyplot as plt\n%matplotlib inline\n\n\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.linear_model import LinearRegression\nfrom sklearn.model_selection import  train_test_split\nfrom sklearn.metrics import r2_score\nfrom sklearn.linear_model import LogisticRegression\nfrom sklearn.tree import DecisionTreeClassifier\nfrom sklearn.metrics import classification_report, confusion_matrix, accuracy_score\nfrom imblearn.over_sampling import SMOTE\nfrom sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold\n'

In [5]:
# Leemos los archivos CSV
file1 = 'C:\\Users\\felip\\Downloads\\M6\\PI_1\\modeloDeRecomendaciones\\datasets\\movies_dataset.csv'
file2 = 'C:\\Users\\felip\\Downloads\\M6\\PI_1\\modeloDeRecomendaciones\\datasets\\credits.csv'
data_movies=pd.read_csv(file1, encoding='UTF-8', sep=',')
data_credits=pd.read_csv(file2, encoding='UTF-8', sep=',')



In [6]:
data_movies.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [7]:
data_credits.head()


,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


In [8]:
#Crear credits_df_cast con las columnas "cast" y "id"
credits_cast = data_credits[['cast', 'id']].copy()

#Crear credits_df_crew con las columnas "crew" y "id"
credits_crew = data_credits[['crew', 'id']].copy()

In [9]:
credits_crew.dropna(inplace=True)
credits_crew.drop_duplicates(inplace=True)


In [10]:
def expandcolumns(df, columns):
    for column in columns:
        df[column] = df[column].apply(lambda x: ast.literal_eval(x) if pd.notnull(x) else []) #Convierte el contenido de cada celda en la columna column de df de una cadena de texto que representa una lista de diccionarios a una lista de diccionarios Python, utilizando ast.literal_eval. Si el valor es nulo (pd.notnull(x)), se devuelve una lista vacía.
        # Expande la columna column en filas, de modo que cada elemento de la lista de diccionarios se convierte en una fila separada en df
        df = df.explode(column)
        # Normaliza la columna column después de expandirla, lo que significa que convierte cada diccionario dentro de la lista en una serie de columnas.
        col_df = pd.json_normalize(df[column])
        # Renombra las columnas normalizadas agregando el nombre original de la columna como prefijo, para evitar conflictos si hay nombres de columnas duplicados en diferentes listas de diccionarios.
        col_df = col_df.add_prefix(f'{column}')
        # Elimina la columna original column del DataFrame df y la reemplaza con las columnas normalizadas (col_df), asegurándose de que el índice del DataFrame se reinicie para evitar problemas con el índice de las filas.
        df = df.drop(columns=[column]).reset_index(drop=True).join(col_df)
    return df

columns_to_expand = ['crew']

#Expandir las columnas de listas de diccionarios
df_crew = expandcolumns(credits_crew, columns_to_expand)

In [11]:
df_crew.head()

,id,crewcredit_id,crewdepartment,crewgender,crewid,crewjob,crewname,crewprofile_path
0,862,52fe4284c3a36847f8024f49,Directing,2.0,7879.0,Director,John Lasseter,/7EdqiNbr4FRjIhKHyPPdFfEEEFG.jpg
1,862,52fe4284c3a36847f8024f4f,Writing,2.0,12891.0,Screenplay,Joss Whedon,/dTiVsuaTVTeGmvkhcyJvKp2A5kr.jpg
2,862,52fe4284c3a36847f8024f55,Writing,2.0,7.0,Screenplay,Andrew Stanton,/pvQWsu0qc8JFQhMVJkTHuexUAa1.jpg
3,862,52fe4284c3a36847f8024f5b,Writing,2.0,12892.0,Screenplay,Joel Cohen,/dAubAiZcvKFbboWlj7oXOkZnTSu.jpg
4,862,52fe4284c3a36847f8024f61,Writing,0.0,12893.0,Screenplay,Alec Sokolow,/v79vlRYi94BZUQnkkyznbGUZLjT.jpg


In [12]:
df_crew.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464698 entries, 0 to 464697
Data columns (total 8 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                464698 non-null  int64  
 1   crewcredit_id     463927 non-null  object 
 2   crewdepartment    463927 non-null  object 
 3   crewgender        463927 non-null  float64
 4   crewid            463927 non-null  float64
 5   crewjob           463927 non-null  object 
 6   crewname          463927 non-null  object 
 7   crewprofile_path  95026 non-null   object 
dtypes: float64(2), int64(1), object(5)
memory usage: 28.4+ MB


In [13]:
df_crew = df_crew[['id','crewjob','crewname']]

In [14]:
df_crew.head()

,id,crewjob,crewname
0,862,Director,John Lasseter
1,862,Screenplay,Joss Whedon
2,862,Screenplay,Andrew Stanton
3,862,Screenplay,Joel Cohen
4,862,Screenplay,Alec Sokolow


In [15]:
df_directors = df_crew[df_crew['crewjob'] == 'Director']

In [16]:
df_directors.head()

,id,crewjob,crewname
0,862,Director,John Lasseter
109,8844,Director,Joe Johnston
122,15602,Director,Howard Deutch
126,31357,Director,Forest Whitaker
141,11862,Director,Charles Shyer


In [17]:
columns_to_expand = ['cast']

#Expandir las columnas de listas de diccionarios
df_cast = expandcolumns(credits_cast, columns_to_expand)

In [18]:
df_cast.head()

,id,castcast_id,castcharacter,castcredit_id,castgender,castid,castname,castorder,castprofile_path
0,862,14.0,Woody (voice),52fe4284c3a36847f8024f95,2.0,31.0,Tom Hanks,0.0,/pQFoyx7rp09CJTAb932F2g8Nlho.jpg
1,862,15.0,Buzz Lightyear (voice),52fe4284c3a36847f8024f99,2.0,12898.0,Tim Allen,1.0,/uX2xVf6pMmPepxnvFWyBtjexzgY.jpg
2,862,16.0,Mr. Potato Head (voice),52fe4284c3a36847f8024f9d,2.0,7167.0,Don Rickles,2.0,/h5BcaDMPRVLHLDzbQavec4xfSdt.jpg
3,862,17.0,Slinky Dog (voice),52fe4284c3a36847f8024fa1,2.0,12899.0,Jim Varney,3.0,/eIo2jVVXYgjDtaHoF19Ll9vtW7h.jpg
4,862,18.0,Rex (voice),52fe4284c3a36847f8024fa5,2.0,12900.0,Wallace Shawn,4.0,/oGE6JqPP2xH4tNORKNqxbNPYi7u.jpg


In [19]:
df_cast.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 564892 entries, 0 to 564891
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   id                564892 non-null  int64  
 1   castcast_id       562474 non-null  float64
 2   castcharacter     562474 non-null  object 
 3   castcredit_id     562474 non-null  object 
 4   castgender        562474 non-null  float64
 5   castid            562474 non-null  float64
 6   castname          562474 non-null  object 
 7   castorder         562474 non-null  float64
 8   castprofile_path  388618 non-null  object 
dtypes: float64(4), int64(1), object(4)
memory usage: 38.8+ MB


In [57]:
df_actors = df_cast[['id','castname']]

In [58]:
df_actors.head()

,id,castname
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles
3,862,Jim Varney
4,862,Wallace Shawn


### Limpieza y transformación

In [22]:
# Eliminamos registros mal cargados y verificamos

data_movies = data_movies[data_movies['imdb_id'] != '0']
data_movies[(data_movies['imdb_id']=='0')]

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count


In [23]:
data_movies.isnull().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   3
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      3
runtime                    260
spoken_languages             3
status                      84
tagline                  25051
title                        3
video                        3
vote_average                 3
vote_count                   3
dtype: int64

In [24]:
# Borramos columnas solicitdas en la consigna.

data_movies.drop(columns=['video','imdb_id','adult','original_title','poster_path','homepage'], inplace=True)

In [25]:
data_movies.isnull().sum()

belongs_to_collection    40972
budget                       0
genres                       0
id                           0
original_language           11
overview                   954
popularity                   3
production_companies         3
production_countries         3
release_date                87
revenue                      3
runtime                    260
spoken_languages             3
status                      84
tagline                  25051
title                        3
vote_average                 3
vote_count                   3
dtype: int64

In [26]:
# Borramos y verificamos los registros que tienen nulos en el campo 'release_date' 

data_movies.dropna(subset=['release_date'], inplace=True)
data_movies.isnull().sum()

belongs_to_collection    40888
budget                       0
genres                       0
id                           0
original_language           11
overview                   941
popularity                   0
production_companies         0
production_countries         0
release_date                 0
revenue                      0
runtime                    246
spoken_languages             0
status                      80
tagline                  24978
title                        0
vote_average                 0
vote_count                   0
dtype: int64

In [27]:
# Reemplazamos nulos por cero en los campos 'revenue' y 'budget'.

data_movies['revenue'].fillna(0, inplace=True)
data_movies['budget'].fillna(0, inplace=True)

In [230]:
# Modificamos el tipo de valor en algunos campos.

data_movies['budget'] = data_movies['budget'].astype(float)
data_movies['popularity'] = data_movies['popularity'].astype(float)
data_movies['id'] = data_movies['id'].astype(str)
df_directors['id'] = df_directors['id'].astype(str)
df_actors['id'] = df_cast['id'].astype(str)

In [60]:
# Creamos nueva columna 'return'.

data_movies['return'] = data_movies['revenue'] / data_movies['budget'].replace(0, 1)

In [63]:
# Modificamos el tipo de dato del campo 'release_date' de object a datetime

data_movies['release_date'] = pd.to_datetime(data_movies['release_date'], errors='coerce').dt.strftime('%Y-%m-%d')

In [62]:
# Creamos un nuevo campo 'release_year'

data_movies['release_year'] = pd.to_datetime(data_movies['release_date']).dt.year


In [65]:
# Suprimimos espacios en blanco en el campo 'title'.

data_movies['title'] = data_movies['title'].str.strip()


In [35]:
#data_movies.info()


In [66]:
# Verificamos filas duplicadas
data_movies.duplicated().sum()

17

In [67]:
# Borramos y volvemos a verificar
data_movies.drop_duplicates(inplace=True)
data_movies.duplicated().sum()

0

In [69]:
# Reemplazamos nulos por cero en campo 'popularity'.

data_movies['popularity'].fillna(0, inplace=True)

In [70]:
# Unimos el campo de directores al data_movies. 

df_movies = data_movies.merge(df_directors, on='id', how='left')

In [71]:
df_movies.head(2)

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,spoken_languages,status,tagline,title,vote_average,vote_count,return,release_year,crewjob,crewname
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,7.7,5415.0,12.451801,1995,Director,John Lasseter
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035,1995,Director,Joe Johnston


In [72]:
df_movies.rename(columns={'crewname': 'director'}, inplace=True)


In [86]:
df_movies['release_date']= pd.to_datetime(data_movies['release_date'],format= '%Y-%m-%d')

In [235]:
# Reagrupamos el dataframe df_actors para luego unirlo al principal

df_actors_reagrupado = df_actors.groupby('id')['castname'].apply(list).reset_index()

In [237]:
df_movies_unida = df_movies.merge(df_actors_reagrupado, on='id', how='left')

In [274]:
df_movies_unida.rename(columns={'castname':'actors'}, inplace=True)

In [279]:
df_movies_unida['director'] = df_movies_unida['director'].astype(str)
df_movies_unida['actors'] = df_movies_unida['actors'].astype(str)

### Creación de funciones

In [239]:
# CANTIDAD DE FILMACIONES POR MES SELECCIONADO EN ESPAÑOL

def cantidad_filmaciones_mes(Mes):
      
    # Mapeo de nombres de meses en español a números de mes. Se incluyen AMBAS variantes para SEPTIEMBRE/SETIEMBRE.
    meses = {'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,'septiembre': 9, 'setiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12}
    
    # Convertir el mes ingresado a minúsculas y obtener su número correspondiente
    mes_numero = meses.get(Mes.lower(), None)
    
    if mes_numero is None:
        return f"No se reconoce el mes '{Mes}'. Por favor, ingresa un mes válido en español."
    
    # Filtrar las películas por el mes especificado
    peliculas_mes = df_movies_unida[df_movies_unida['release_date'].dt.month == mes_numero]
    
    # Obtener la cantidad de películas
    cantidad = len(peliculas_mes)
    
    #Devolver el resultado formateado
    return f"{cantidad} {'películas' if cantidad != 1 else 'película'} fueron estrenadas en el mes de {Mes.capitalize()}."




In [251]:
# Probamos la función:
mes_consultado = "setiembre"
resultado = cantidad_filmaciones_mes(mes_consultado)
resultado

'4836 películas fueron estrenadas en el mes de Setiembre.'

In [252]:
# CANTIDAD DE FILMACIONES POR DÍA SELECCIONADO EN ESPAÑOL

def cantidad_filmaciones_dia(Dia):
    
    # Mapeo de nombres de días en español a números de día. Se incluyen variantes sin acento ortográfico.
    dias = {'lunes': 0, 'martes': 1, 'miércoles': 2, 'miercoles': 2, 'jueves': 3, 'viernes': 4, 'sábado': 5, 'sabado': 5,'domingo': 6}
    
    # Convertir el día ingresado a minúsculas y obtener su número correspondiente
    dia_numero = dias.get(Dia.lower(), None)
    
    if dia_numero is None:
        return f"No se reconoce el día '{Dia}'. Por favor, ingresa un día válido en español."
    
    # Filtrar las películas por el día especificado
    peliculas_dia = df_movies_unida[df_movies_unida['release_date'].dt.dayofweek == dia_numero]
    
    # Obtener la cantidad de películas
    cantidad = len(peliculas_dia)
    
    # Devolver el resultado formateado
    return f"{cantidad} {'películas' if cantidad != 1 else 'película'} fueron estrenadas en los días {Dia.title()}."



In [254]:
# Probamos la función:
dia_consultado = "martES"
resultado = cantidad_filmaciones_dia(dia_consultado)
print(resultado)

4640 películas fueron estrenadas en los días Martes.


In [255]:
# TÍTULO, AÑO DE ESTRENO Y POPULARIDAD

def score_titulo(titulo_de_la_filmacion):
        
    # Convertimos el título ingresado a minúsculas para la búsqueda
    titulo_ingresado_lower = titulo_de_la_filmacion.lower()

    # Filtramos la película por el título especificado en minúsculas
    pelicula = df_movies_unida[df_movies_unida['title'].str.lower() == titulo_ingresado_lower]
    
    if pelicula.empty:
        return f"No se encontró información para la película '{titulo_de_la_filmacion}'."
    else:
        # Obtener el título original tal cual está en el dataset, el año de estreno y el score.
        titulo_original = pelicula['title'].iloc[0]
        año_estreno = pelicula['release_date'].dt.year.iloc[0]
        score = pelicula['popularity'].iloc[0]
        return f"La película '{titulo_original}' fue estrenada en el año {año_estreno} con un score/popularidad de {score:.2f}."



In [257]:
# # Probamos la función sin considerar mayúsculas y minúsculas:
titulo_consultado = "shrek"
resultado = score_titulo(titulo_consultado)
print(resultado)

La película 'Shrek' fue estrenada en el año 1989 con un score/popularidad de 17.99.


In [262]:
# TÍTULO, CANTIDAD DE VOTOS Y VALOR PROMEDIO arriba de 2000 VOTOS

def votos_titulo(titulo_de_la_filmacion):
        
    # Convertimos el título ingresado a minúsculas para la búsqueda
    titulo_ingresado_lower = titulo_de_la_filmacion.lower()

    # Filtramos la película por el título especificado en minúsculas
    pelicula = df_movies_unida[df_movies_unida['title'].str.lower() == titulo_ingresado_lower]
        
    if pelicula.empty:
        return f"No se encontró información para la película '{titulo_de_la_filmacion}'."
    else:
        votos = int(pelicula['vote_count'].iloc[0])
        if votos < 2000:
            return f"La película '{titulo_de_la_filmacion}' fue estrenada en el año {pelicula['release_date'].dt.year.iloc[0]}. La misma cuenta con menos de 2000 valoraciones."
        else:
            titulo_original = pelicula['title'].iloc[0]
            promedio = round(pelicula['popularity'].iloc[0], 2)
            return f"La película '{titulo_original}' fue estrenada en el año {pelicula['release_date'].dt.year.iloc[0]}. Tiene un total de {votos} valoraciones, con un promedio de {promedio}."

In [266]:
# Probamos la función sin considerar mayúsculas y minúsculas:
titulo_consultado = "sabrina"
resultado = votos_titulo(titulo_consultado)
print(resultado)

La película 'sabrina' fue estrenada en el año 1995. La misma cuenta con menos de 2000 valoraciones.


In [282]:
# RETORNO Y CANTIDAD DE PELICULAS POR ACTOR

def get_actor(nombre_actor):
    
    nombre_actor_lower = nombre_actor.lower()
    peliculas_actor = df_movies_unida[df_movies_unida['actors'].str.lower().str.contains(nombre_actor_lower, na=False) & (~df_movies_unida['director'].str.lower().str.contains(nombre_actor_lower, na=False))]
    if peliculas_actor.empty:
        return f"No se encontró información para el actor '{nombre_actor}'."
    else:
        cantidad_peliculas = len(peliculas_actor)
        retorno_promedio = peliculas_actor['revenue'].mean()
        return f"{nombre_actor} ha participado en {cantidad_peliculas} {'filmaciones' if cantidad_peliculas != 1 else 'filmación'}, con un retorno promedio de ${retorno_promedio:.2f}."



In [283]:
# Probamos la función:
actor_consultado = "tom hardy"
resultado = get_actor(actor_consultado)
print(resultado)

tom hardy ha participado en 43 filmaciones, con un retorno promedio de $92708833.09.


In [284]:
# RETORNO TOTAL POR DIRECTOR Y DETALLE DE PELICULAS

def get_director(nombre_director):
    # Convertir el nombre del director ingresado a minúsculas para la búsqueda
    nombre_director_lower = nombre_director.lower()

    # Filtrar las películas por el director especificado en minúsculas
    peliculas_director = df_movies_unida[df_movies_unida['director'].str.lower() == nombre_director_lower]

    if peliculas_director.empty:
        return f"No se encontró información para el director '{nombre_director}'."
    else:
        # Obtener la cantidad de películas y el total de retorno
        cantidad_peliculas = len(peliculas_director)
        retorno_total = round(peliculas_director['return'].sum(), 2)

        # Obtener detalles de las películas (nombre, fecha de lanzamiento, retorno, costo, ganancia)
        detalles_peliculas = []
        for _, row in peliculas_director.iterrows():
            nombre_pelicula = row['title']
            fecha_lanzamiento = row['release_date'] #.strftime("%Y-%m-%d")
            retorno_individual = row['return']
            costo = row['budget']
            ganancia = row['revenue']
            detalles_peliculas.append(f"{nombre_pelicula} (Lanzamiento: {fecha_lanzamiento}, Retorno: {retorno_individual:2f}, Costo: {costo}, Ganancia: {ganancia})")

        # Crear una lista con cada película y sus detalles en líneas separadas
        peliculas_formateadas = "\n".join(detalles_peliculas)

        return f"El director '{nombre_director}' ha dirigido {cantidad_peliculas} {'películas' if cantidad_peliculas != 1 else 'película'}, con un retorno total de {retorno_total}.\nDetalles de las películas:\n{peliculas_formateadas}"
    

In [285]:
# Probamos la función:
director_consultado = "Tom hanks"
resultado = get_director(director_consultado)
print(resultado)

El director 'Tom hanks' ha dirigido 3 películas, con un retorno total de 34585417.21.
Detalles de las películas:
That Thing You Do! (Lanzamiento: 1996-09-27 00:00:00, Retorno: 34585416.000000, Costo: 0.0, Ganancia: 34585416.0)
Larry Crowne (Lanzamiento: 2011-08-17 00:00:00, Retorno: 1.205346, Costo: 30000000.0, Ganancia: 36160375.0)
Band of Brothers (Lanzamiento: NaT, Retorno: 0.000000, Costo: 125000000.0, Ganancia: 0.0)


In [286]:
# Exportamos el dataframe como .CSV

df_movies_unida.to_parquet('C:\\Users\\felip\\Desktop\\Proyecto1\\df_movies.parquet', index=False)

In [56]:
# data_movies.shape

In [228]:
df_actors.head()

,id,castname
0,862,Tom Hanks
1,862,Tim Allen
2,862,Don Rickles
3,862,Jim Varney
4,862,Wallace Shawn


In [231]:
result = df_actors.groupby('id')['castname'].apply(list).reset_index()

In [232]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45432 entries, 0 to 45431
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        45432 non-null  object
 1   castname  45432 non-null  object
dtypes: object(2)
memory usage: 710.0+ KB


In [233]:
junto = df_movies.merge(result, on='id', how='left')

In [234]:
junto.head()

,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,release_date,...,status,tagline,title,vote_average,vote_count,return,release_year,crewjob,director,castname
0,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,...,Released,NaN,Toy Story,7.7,5415.0,1.245180e+01,1995,Director,John Lasseter,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney..."
1,NaN,65000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",8844,en,When siblings Judy and Peter discover an encha...,17.015539,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,...,Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,4.043035e+00,1995,Director,Joe Johnston,"[Robin Williams, Jonathan Hyde, Kirsten Dunst,..."
2,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0.0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",15602,en,A family wedding reignites the ancient feud be...,11.712900,"[{'name': 'Warner Bros.', 'id': 6194}, {'name'...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,0.000000e+00,1995,Director,Howard Deutch,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sop..."
3,NaN,16000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,[{'name': 'Twentieth Century Fox Film Corporat...,"[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-22,...,Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,5.090760e+00,1995,Director,Forest Whitaker,"[Whitney Houston, Angela Bassett, Loretta Devi..."
4,"{'id': 96871, 'name': 'Father of the Bride Col...",0.0,"[{'id': 35, 'name': 'Comedy'}]",11862,en,Just when George Banks has recovered from his ...,8.387519,"[{'name': 'Sandollar Productions', 'id': 5842}...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-02-10,...,Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,7.657891e+07,1995,Director,Charles Shyer,"[Steve Martin, Diane Keaton, Martin Short, Kim..."


In [273]:
df_movies_unida.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49822 entries, 0 to 49821
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   belongs_to_collection  5169 non-null   object        
 1   budget                 49822 non-null  float64       
 2   genres                 49822 non-null  object        
 3   id                     49822 non-null  object        
 4   original_language      49807 non-null  object        
 5   overview               48819 non-null  object        
 6   popularity             49822 non-null  float64       
 7   production_companies   49822 non-null  object        
 8   production_countries   49822 non-null  object        
 9   release_date           45359 non-null  datetime64[ns]
 10  revenue                49822 non-null  float64       
 11  runtime                49562 non-null  float64       
 12  spoken_languages       49822 non-null  object        
 13  s

In [280]:
def get_actor_junto(nombre_actor):
    
    nombre_actor_lower = nombre_actor.lower()
    peliculas_actor = df_movies_unida[df_movies_unida['actors'].str.lower().str.contains(nombre_actor_lower, na=False) & (~df_movies_unida['director'].str.lower().str.contains(nombre_actor_lower, na=False))]
    if peliculas_actor.empty:
        return f"No se encontró información para el actor '{nombre_actor}'."
    else:
        cantidad_peliculas = len(peliculas_actor)
        retorno_promedio = peliculas_actor['revenue'].mean()
        return f"{nombre_actor} ha participado en {cantidad_peliculas} {'filmaciones' if cantidad_peliculas != 1 else 'filmación'}, con un retorno promedio de ${retorno_promedio:.2f}."

In [281]:
# Probamos la función:
actor_consultado = "ben affleck"
resultado = get_actor_junto(actor_consultado)
print(resultado)

ben affleck ha participado en 49 filmaciones, con un retorno promedio de $107216007.78.
